Image resizing
https://blog.roboflow.com/you-might-be-resizing-your-images-incorrectly/

We have two set of images
Mike : 3712x2088
Neil : 5120x3840

We need to resize
1. resize the image
2. crop to be square

recreate folder structure and resize

In [14]:
import matplotlib.pyplot as plt
import skimage.transform
import scipy.misc 
import imageio
import math
import numpy as np
import os

In [15]:
IMG_WIDTH = 500
MAKE_SQUARE = False

#read paths
#MIKE_ORIGINALS = r'C:\DS Project\Images\Mike\test'
MIKE_ORIGINALS = r'C:\DS Project\Images\Mike\Original'
NEIL_ORIGINALS = r'C:\DS Project\Images\Neil\Original'

MIKE_TRACTORS = r'C:\DS Project\Images\Sorted_Tractors\Mike'
NEIL_TRACTORS = r'C:\DS Project\Images\Sorted_Tractors\Neil'

#save paths
MIKE_ALL = r'C:\DS Project\_processed\Images\resized_'+ str(IMG_WIDTH) + ('_sq' if MAKE_SQUARE == True else '_') + r'\all\Mike'
NEIL_ALL = r'C:\DS Project\_processed\Images\resized_'+ str(IMG_WIDTH) + ('_sq' if MAKE_SQUARE == True else '_') + r'\all\Neil'

TRACTORS_MIKE_TO = r'C:\DS Project\_processed\Images\resized_'+ str(IMG_WIDTH) + ('_sq' if MAKE_SQUARE == True else '_') + r'\tractors\Mike\to'
TRACTORS_NEIL_TO = r'C:\DS Project\_processed\Images\resized_'+ str(IMG_WIDTH) + ('_sq' if MAKE_SQUARE == True else '_') + r'\tractors\Neil\to'

TRACTORS_MIKE_FROM = r'C:\DS Project\_processed\Images\resized_'+ str(IMG_WIDTH) + ('_sq' if MAKE_SQUARE == True else '_') + r'\tractors\Mike\from'
TRACTORS_NEIL_FROM = r'C:\DS Project\_processed\Images\resized_'+ str(IMG_WIDTH) + ('_sq' if MAKE_SQUARE == True else '_') + r'\tractors\Neil\from'
        

In [16]:
def bound_image_dim(image, min_size=None, max_size=None):
    
    if (max_size is not None) and \
       (min_size is not None) and \
       (max_size < min_size):
        raise ValueError('`max_size` must be >= to `min_size`')
        
    dtype = image.dtype
    (height, width, *_) = image.shape
    
    # scale the same for both height and width for fixed aspect ratio resize
    scale = 1
    
    # bound the smallest dimension to the min_size
    if min_size is not None:
        image_min = min(height, width)
        scale = max(1, min_size / image_min)
    
    # next, bound the largest dimension to the max_size
    # this must be done after bounding to the min_size
    if max_size is not None:
        image_max = max(height, width)
        if round(image_max * scale) > max_size:
            scale = max_size / image_max
    if scale != 1:
        image = skimage.transform.resize(
            image, (round(height * scale), round(width * scale)),
            order=1,
            mode='constant',
            preserve_range=True)
    return image.astype(dtype)


def square_pad_image(image, size):
    
    #get dimensions
    (height, width, *_) = image.shape
    
    # check 
    if (size < height) or (size < width):
        raise ValueError('`size` must be >= to image height and image width')
        
    pad_height = (size - height) / 2
    pad_top = math.floor(pad_height)
    pad_bot = math.ceil(pad_height)
    pad_width = (size - width) / 2
    pad_left = math.floor(pad_width)
    pad_right = math.ceil(pad_width)
    return np.pad(
        image, ((pad_top, pad_bot), (pad_left, pad_right), (0, 0)),
        mode='constant')


def createFolders():
    
    try:
        os.makedirs(MIKE_ALL)
    except OSError:
        print ("Creation of the directory %s failed" % MIKE_ALL)
    else:
        print ("Successfully created the directory %s " % MIKE_ALL)

    try:
        os.makedirs(NEIL_ALL)
    except OSError:
        print ("Creation of the directory %s failed" % NEIL_ALL)
    else:
        print ("Successfully created the directory %s " % NEIL_ALL)

    try:
        os.makedirs(TRACTORS_MIKE_TO)
    except OSError:
        print ("Creation of the directory %s failed" % TRACTORS_MIKE_TO)
    else:
        print ("Successfully created the directory %s " % TRACTORS_MIKE_TO)

    try:
        os.makedirs(TRACTORS_NEIL_TO)
    except OSError:
        print ("Creation of the directory %s failed" % TRACTORS_NEIL_TO)
    else:
        print ("Successfully created the directory %s " % TRACTORS_NEIL_TO)
  
    try:
        os.makedirs(TRACTORS_MIKE_FROM)
    except OSError:
        print ("Creation of the directory %s failed" % TRACTORS_MIKE_FROM)
    else:
        print ("Successfully created the directory %s " % TRACTORS_MIKE_FROM)

    try:
        os.makedirs(TRACTORS_NEIL_FROM)
    except OSError:
        print ("Creation of the directory %s failed" % TRACTORS_NEIL_FROM)
    else:
        print ("Successfully created the directory %s " % TRACTORS_NEIL_FROM)
    
    
def processImages(Path, toFilePath_To, toFilePath_From):
      
    toImages = Path + r'\to'
    fromImages = Path + r'\from'
    
    #loop round each 'to' images, resixe and save on 'to folder
    for filename in os.listdir(toImages):
       
        filepath = toImages + os.sep + filename
        origImg = imageio.imread(filepath)

        h,w,c = origImg.shape
    
        resizedImg = bound_image_dim(origImg,0,IMG_WIDTH)
            
        if (MAKE_SQUARE == True):
                squaredImg = square_pad_image(resizedImg, IMG_WIDTH)
                plt.imsave(toFilePath_To + r'\\' + filename, squaredImg)
                
        else:
                plt.imsave(toFilePath_To + r'\\' + filename, resizedImg)
            
        print ("processed..", toFilePath_To + r'\\' + filename)
    
    #loop round each 'from' image, resize and save in 'from' folder
    for filename in os.listdir(fromImages):
       
        filepath = fromImages + os.sep + filename
        origImg = imageio.imread(filepath)

        h,w,c = origImg.shape
    
        resizedImg = bound_image_dim(origImg,0,IMG_WIDTH)
            
        if (MAKE_SQUARE == True):
                squaredImg = square_pad_image(resizedImg, IMG_WIDTH)
                plt.imsave(toFilePath_From + r'\\' + filename, squaredImg)
                
        else:
                plt.imsave(toFilePath_From + r'\\' + filename, resizedImg)
            
        print ("processed..", toFilePath_From + r'\\' + filename)
     
  
         

In [17]:
#create folders
createFolders()
print("folders created")

#process mike orginals
#processImages(MIKE_ORIGINALS,MIKE_ALL)
#print("processed mike orginals")

#process neil originals
#processImages(NEIL_ORIGINALS, NEIL_ALL)
#print("processed neil orginals")

#process mike tractors
processImages(MIKE_TRACTORS, TRACTORS_MIKE_TO, TRACTORS_MIKE_FROM)
print("processed mike tractors")

#process neil tractors
processImages(NEIL_TRACTORS, TRACTORS_NEIL_TO, TRACTORS_NEIL_FROM)
print("processed neil tractors")

print("all images resized")



Successfully created the directory C:\DS Project\_processed\Images\resized_500_\all\Mike 
Successfully created the directory C:\DS Project\_processed\Images\resized_500_\all\Neil 
Successfully created the directory C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to 
Successfully created the directory C:\DS Project\_processed\Images\resized_500_\tractors\Neil\to 
Successfully created the directory C:\DS Project\_processed\Images\resized_500_\tractors\Mike\from 
Successfully created the directory C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from 
folders created
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\01Mk0132.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\01Mk0144.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\01Mk0191.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\01Mk0203.JPG
processed.. C:\DS Project\_processed\Images\resized_500_

processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\03Mk0297.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\03Mk0335.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\03Mk0386.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\03Mk0416.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\03Mk0434.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\03Mk0474.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\03Mk0528.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\03Mk0561.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\03Mk0683.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\03Mk0742.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\03Mk0805.JPG
processed.. C:\DS Project\_proce

processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk0595.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk0605.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk0609.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk0617.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk0621.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk0730.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk2189.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk2204.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk2213.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk2224.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk2239.JPG
processed.. C:\DS Project\_proce

processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk3290.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk3305.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk3325.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk3328.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk3336.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk3344.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk3355.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk3363.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk3368.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk3376.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\04Mk3393.JPG
processed.. C:\DS Project\_proce

processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\06Mk0671.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\06Mk0674.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\06Mk0684.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\06Mk0689.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\06Mk0708.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\06Mk0735.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\06Mk0760.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\06Mk0766.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\06Mk0780.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\06Mk0794.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Mike\to\\06Mk0804.JPG
processed.. C:\DS Project\_proce

...

processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\14PT1499.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1502.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1503.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1505.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1507.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1510.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1511.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1513.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1516.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1517.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1519.JPG
processed.

processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1683.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1686.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1687.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1690.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1691.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1694.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1697.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1699.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1701.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1703.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1707.JPG
processed.

processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1972.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1977.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1980.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1981.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1982.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1985.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1989.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1990.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1992.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1993.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\15PT1994.JPG
processed.

processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1538.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1540.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1542.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1545.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1546.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1548.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1550.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1553.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1554.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1557.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1559.JPG
processed.

processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1834.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1835.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1836.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1847.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1849.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1850.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1851.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1866.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1908.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1910.JPG
processed.. C:\DS Project\_processed\Images\resized_500_\tractors\Neil\from\\20PT1923.JPG
processed.